# AKSSF Anadromous Waters Catalog Data Join
Use spatial join to add AWC_CODE, Species and LSTAGE information to AKSSF catchments.  Add new field to concatenate species+lstage and field ot designate salmon spawning habitat presence or absence.

In [1]:
import os, arcpy, sys,datetime, traceback
import time

arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

print('imports complete')
print(f'{("-"*100)}')
print(f'sys paths {sys.path}')
print(f'{("-"*100)}')
print(f'Python Environment set to - {sys.base_exec_prefix}')
print(f'{("-"*100)}')
print (datetime.datetime.now())
outdir = os.path.dirname(os.getcwd())
print(f'CSV table output directory set to {outdir}')

imports complete
----------------------------------------------------------------------------------------------------
sys paths ['C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\data_preparation\\sensitivity_drivers\\geomorphology', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\Resources\\ArcPy', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\python37.zip', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\DLLs', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3', '', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin', 'C:\\Users\\dwmerrigan\\A

## Functions

In [2]:
# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value
# Function to remove parenthesis from user inputs
def replace_all(userinput, dic):
    for i, j in dic.items():
        userinput = userinput.replace(i, j)
    return userinput

# Getnull rows from numpy array
def getnull(cat_ID_con):
    nullRows = []
    nullRows.append(cat_ID_con)
    return True

#Generate unique column names
def uniquify(df_final):
    seen = set()
    for item in df_final:
        fudge = 1
        newitem = item
        while newitem in seen:
            fudge += 1
            newitem = "{}_{}".format(item, fudge)
        yield newitem
        seen.add(newitem)

## Section 1
### Set input datasets and output locations
User to input paths for necessary input data and output locations



In [25]:
# Get user inputs
# Used to format user inputs
inputDict = {"'":"",'"':""}# Specify path to AKSSF parent directory
while True:
    try:
        userinput = replace_all((input('Input path to temp site catchments fc.\nLeave blank and hit enter to use the default location.\nDefault = D:\\GIS\\AKSSF_land_met\\AKSSF_land_met.gdb\\all_cats_int_merge') or 'D:\\GIS\\AKSSF_land_met\\AKSSF_land_met.gdb\\all_cats_int_merge'),inputDict)
        if not arcpy.Exists(userinput):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            catsint = userinput
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'AKSSF temp site catchments {catsint}\n {"-"*100}')

# Specify path to AWC events fc
while True:
    try:
        userinput2 = replace_all((input('Input path to awc events feature class or shapefile.\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs') or "D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs"), inputDict)
        if not arcpy.Exists(userinput2):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            awc_events = userinput2
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'AWC events feature class set to {awc_events}\n {"-"*100}')

# Specify path to AWC events fc
while True:
    try:
        userinput2 = replace_all((input('Input path to output gdb.\nLeave blank and hit enter to use the default location.\nDefault = C:\\Users\\dwmerrigan\\Documents\\GitHub\\ArcGIS_Default\\ArcGIS_Default.gdb') or "C:\\Users\\dwmerrigan\\Documents\\GitHub\\ArcGIS_Default\\ArcGIS_Default.gdb"), inputDict)
        if not arcpy.Exists(userinput2):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            outputgdb = userinput2
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'Output GDB set to {outputgdb}\n {"-"*100}')



AKSSF temp site catchments D:\GIS\AKSSF_land_met\AKSSF_land_met.gdb\all_cats_int_merge
 ----------------------------------------------------------------------------------------------------
AWC events feature class set to D:\Basedata\AWC\AWC_2021_SpeciesEvents.gdb\awcEventArcs
 ----------------------------------------------------------------------------------------------------
Output GDB set to C:\Users\dwmerrigan\Documents\GitHub\ArcGIS_Default\ArcGIS_Default.gdb
 ----------------------------------------------------------------------------------------------------


## Join awc data to catchments via spatial join
 * Add field 'sp_ls' to record species and l_stage comma seperated concatenations and field 'sa_sp_hab' to record presence absence of salmon spawning habitat

In [51]:
import time, datetime
arcpy.env.overwriteOutput = True
# Parameters
joinopt = "JOIN_ONE_TO_ONE"
jointype = "KEEP_ALL"
matchopt = "INTERSECT"
s_rad = "1 Centimeters"

processStart = time.time()

awcjoin = arcpy.analysis.SpatialJoin(catsint, awc_events, os.path.join(outputgdb,'cats_int_awc_spJoin'), joinopt, jointype, r'SourceFC "SourceFeatureClass" true true false 20 Text 0 0,First,#,catsint,SourceFC,0,20;AreaSqKm "AreaSqKm" true true false 8 Double 0 0,First,#,catsint,AreaSqKm,-1,-1;cat_ID_con "cat_ID_con" true true false 255 Text 0 0,First,#,catsint,cat_ID_con,0,255;MERGE_SRC "MERGE_SRC" true true false 255 Text 0 0,First,#,catsint,MERGE_SRC,0,255;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,catsint,Shape_Length,-1,-1,awc_events,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,catsint,Shape_Area,-1,-1;region "region" true true false 255 Text 0 0,First,#,catsint,region,0,255,awc_events,REGION,0,20;AWC_CODE "AWC_CODE" true false false 500 Text 0 0,Join,",",awc_events,AWC_CODE,0,70;SPECIES "SPECIES" true false false 500 Text 0 0,Join,",",awc_events,SPECIES,0,2;LSTAGE "LSTAGE" true false false 500 Text 0 0,Join,",",awc_events,LSTAGE,0,1', matchopt,s_rad, '')

arcpy.AddField_management(awcjoin,'sp_ls','TEXT')
arcpy.AddField_management(awcjoin,'sa_sp_hab','SHORT')
arcpy.AddField_management(awcjoin,'mult_awc','SHORT')
arcpy.AddField_management(awcjoin,'AWC_CODE_qc','TEXT',"","",500)

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()
print(f'Process complete')

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print(f'{"*"*100}')


Process complete
Process completed at 2022-02-16 13:57 (Elapsed time: 0:02:42)
****************************************************************************************************



## Make dictionary from awc events


In [48]:
awcDict = {}
with arcpy.da.SearchCursor(awc_events, ['AWC_CODE',"SPECIES","LSTAGE"]) as cur:
    for row in cur:
        sp_ls_val  = row[1] + "_" + row[2]
        append_value(awcDict,row[0],sp_ls_val)
awcDict

{'306-70-10700-2002': ['CO_r', 'P_s'],
 '316-20-10100': ['SH_p',
  'CH_p',
  'P_p',
  'S_s',
  'CH_s',
  'S_p',
  'CO_p',
  'DV_p',
  'K_s',
  'K_p'],
 '313-20-10400': ['CO_r', 'CO_p', 'K_r', 'S_p', 'K_p'],
 '283-63-11500': ['P_s', 'CH_s'],
 '305-52-10520': 'P_s',
 '307-16-12800-2025': ['DV_p', 'CO_s', 'P_s'],
 '283-42-10500-2001': ['CO_p', 'CH_p', 'P_p'],
 '306-70-10135': 'CO_r',
 '281-33-10100-2002': ['CH_s', 'P_p', 'P_s', 'CH_p'],
 '314-20-10300-2006': ['DV_r', 'CO_r'],
 '302-50-11470': 'P_s',
 '283-34-10700': ['CH_s', 'S_r', 'CO_p', 'S_s', 'CO_r'],
 '313-10-10340-2013': ['P_p', 'S_s', 'CH_s', 'CO_p', 'CH_p', 'K_s'],
 '315-11-10200-2009-3005': ['K_p', 'CO_p', 'OU_p', 'S_p', 'CH_p'],
 '283-12-11200': 'CH_s',
 '302-60-11100': ['P_s', 'S_s'],
 '275-40-10050': ['CH_s', 'P_s'],
 '303-21-10300': ['CO_s', 'DV_p', 'P_s'],
 '302-21-10300': 'DV_p',
 '312-40-10100-2021-3012': ['S_p', 'S_r', 'S_s', 'CO_r'],
 '306-70-10200': ['P_s', 'DV_p', 'CO_p'],
 '316-20-10310': ['SH_p',
  'CO_p',
  'S_s',
 

## Use update cursor to populate new field


In [59]:
awcjoin = os.path.join(outputgdb,'cats_int_awc_spJoin')
dropfields = ['Join_Count', 'NHDPlusID','catID','Shape','TARGET_FID','SOURCEFC']
keepfields = [f.name for f in arcpy.ListFields(awcjoin) if f.name not in dropfields]
sa_sp_codes = ['CO_s','S_s','K_s', 'CH_s']
print(keepfields)
multirecslist = []
multirecsDict = {}
norecslist = []
sp_lsList = []
awcCodeDict = {}
catAwcDict = {}
curfields = ['cat_ID_con','AWC_CODE','Species', 'LSTAGE', 'sp_ls','sa_sp_hab','mult_awc','AWC_CODE_qc']
c2 = 0
with arcpy.da.UpdateCursor(awcjoin, curfields) as cur:
    for row in cur:
        awcvals = []
        print(f'{(100*"*")}')
        if row[1] is not None:
            for val in row[1].split(','):
                val2 = "'"+str(val)+"'"
                awcvals.append(val)
            awcvals = list(set(awcvals))
            if len(str(awcvals).split(',')) > 1:
                print(f'Multiple awc records({len(str(awcvals).split(","))}) for this catchment ')
                append_value(multirecsDict,row[0],awcvals)
                row[6] = 1
                row[7] = str(awcvals).strip("[]").replace("'","")
            else:
                row[7]=awcvals[0]
                row[6] = 0
            spvals = row[2].split(",")
            lsvals = row[3].split(",")
            snum = len(row[2].split(","))
            cat_spList =[]
            for i in range(snum):
                sp_ls = spvals[i] + '_' + lsvals[i]
                append_value(awcCodeDict,row[1],sp_ls)
                append_value(catAwcDict,row[0],sp_ls)
                sp_lsList.append(sp_ls)
                cat_spList.append(sp_ls)
            print(f'{row[0]} has {snum} species_ls records {cat_spList}')
            print(f'list without duplicate recrds {list(set(cat_spList))}')
            row[4] = str(list(set(cat_spList))).strip("[]").replace("'","")
            if len(set(cat_spList).intersection(sa_sp_codes)) >0:
                print(f'{row[0]} has {set(cat_spList).intersection(sa_sp_codes)} salmon spawning records')
                row[5] = 1
            else:
                print(f'{row[0]} has no spawning habitat recorded')
                row[5] = 0
        else:
            print(f'{row[0]} did not intersect the AWC ')
            row[5] = 0
            row[6] = 0
            norecslist.append(row[0])
            c2+=1
        cur.updateRow(row)
    del(row)
del(cur)

print(f'{c2} catchments did not have AWC species records')

['OBJECTID', 'SourceFC', 'AreaSqKm', 'cat_ID_con', 'MERGE_SRC', 'region', 'AWC_CODE', 'SPECIES', 'LSTAGE', 'Shape_Length', 'Shape_Area', 'sp_ls', 'sa_sp_hab', 'mult_awc', 'AWC_CODE_qc']
****************************************************************************************************
Cook_Inlet_75004300006312 has 5 species_ls records ['DV_p', 'K_s', 'SH_p', 'CO_s', 'DV_r']
list without duplicate recrds ['DV_p', 'SH_p', 'DV_r', 'K_s', 'CO_s']
Cook_Inlet_75004300006312 has {'CO_s', 'K_s'} salmon spawning records
****************************************************************************************************
Cook_Inlet_75004300001906 has 3 species_ls records ['CO_s', 'DV_p', 'S_s']
list without duplicate recrds ['DV_p', 'S_s', 'CO_s']
Cook_Inlet_75004300001906 has {'S_s', 'CO_s'} salmon spawning records
****************************************************************************************************
Multiple awc records(3) for this catchment 
Cook_Inlet_75004300000100 has 13 spec

## Copy and snap points to awc clip